# Video Game Console Analysis

## 1. Introduction

## 2. Data Cleaning

In [2]:
# Import libraries 

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats as st
import math

In [7]:
# Import the data

filepath = './data/games.csv'
try:
    games_df = pd.read_csv(filepath)
except FileNotFoundError as e:
    print(f"The file at [{filepath}] was not found. The following error: {e} occurred.\nTry again!")
else:
    print(f"The file at the filepath: [{filepath}] was imported.")

The file at the filepath: [./data/games.csv] was imported.
